# NASA Earthdata Worldview

https://worldview.earthdata.nasa.gov/?p=geographic&l=SMAP_L4_Emult_Average&t=2018-08-01-T00%3A00%3A00Z&z=3&v=-180,-90,180,90

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import pandas as pd
from imageio import mimwrite
import requests

In [ ]:
def collect_img_binary(date, layer, extent, width, height):
    
    # Create URL to download image
    nasa_url = "https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot"
    nasa_url += "&TIME=%s" % date.strftime('%Y-%m-%d')
    nasa_url += "&BBOX=%s" % ','.join([str(e) for e in extent])
    nasa_url += "&CRS=EPSG:4326"
    nasa_url += "&LAYERS=%s" % layer
    nasa_url += "&FORMAT=image/png"
    nasa_url += "&WIDTH={0:d}&HEIGHT={1:d}".format(width, height)
    
    # Request content and save binary PNG file in variable
    r = requests.get(nasa_url, allow_redirects=True)
    img_binary = r.content
    
    return img_binary

In [ ]:
def recreate_img(img_binary, height):
    
    # Store binary content in PNG file 'temp.png'
    with open('temp.png', 'wb') as f:
        f.write(img_binary)

    # Open PNG file and extract data value for each RGBA pixel
    im_frame = Image.open('temp.png')
    data = np.array(im_frame.getdata()).reshape(height, -1, 4)
    
    return data

In [ ]:
def collect_dataset(start_date, end_date, layer, extent=(27, -10, 63, 43),
                    width=400, frequency='1MS'):
    
    # Compute height based on width and the bounding box of the image
    relation = float(extent[2] - extent[0]) / (extent[3] - extent[1])
    height = int(width * relation)
    
    timeline = pd.date_range(start_date, end_date, freq=frequency)
    
    dataset = []
    
    for t in timeline:
        
        img_binary = collect_img_binary(t.date(), layer, extent, width, height)
    
        data = recreate_img(img_binary, height)

        data = data[..., :3].astype('uint8')

        dataset.append(data)

        print(t.strftime('%Y-%m-%d'))
    
    return np.array(dataset)

In [ ]:
def img_has_content(data, thresh=0.15):
    rows_with_content = np.where(np.sum(data, axis=(-2, -1)))
    return np.max(rows_with_content) > data.shape[1]*thresh

# Test functions

In [ ]:
layer = 'VIIRS_SNPP_CorrectedReflectance_TrueColor'

In [ ]:
start_date = '2018-01-01'
end_date = '2018-12-31'

In [ ]:
extent=(-80, -160, 80, 180)
width=1280
frequency='12MS'

In [ ]:
dataset = collect_dataset(start_date, end_date, layer,
                          extent, width, frequency)

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(dataset[0]);

In [ ]:
layers=['MODIS_Aqua_Chlorophyll_A',
        'MODIS_Combined_Thermal_Anomalies_All',
        'MODIS_Terra_Chlorophyll_A',
        'MODIS_Terra_EVI_8Day',
        'SMAP_L1_Passive_Enhanced_Brightness_Temp_Fore_H',
        'SMAP_L2_Passive_Enhanced_Day_Soil_Moisture_Option1',
        'SMAP_L4_Emult_Average',
        'SMAP_L4_Mean_Heterotrophic_Respiration',
        'SMAP_L4_Mean_Net_Ecosystem_Exchange',
        'SMAP_L4_Soil_Temperature_Layer_1',
        'SMAP_L4_Uncertainty_Mean_Net_Ecosystem_Exchange',
        'VIIRS_SNPP_CorrectedReflectance_TrueColor'
        ]
layers=['VIIRS_SNPP_CorrectedReflectance_TrueColor'
        ]

In [ ]:
start_date = '2018-01-01'
end_date = '2018-12-31'

extent=(-60, -160, 80, 180)
width=1280
frequency='1MS'

for l in layers:
    print('\n', l)
    dataset = collect_dataset(start_date, end_date, l, extent, width, frequency)
    mimwrite('gif_%s.gif' % l, dataset, format='gif', fps=6)

In [ ]:
start_date = '2018-01-01'
end_date = '2018-04-01'
width=500
frequency='1D'

In [ ]:
layer = 'VIIRS_SNPP_CorrectedReflectance_TrueColor'
extent = (35, -15, 60, 20)
dataset = collect_dataset(start_date, end_date, layer, extent, width, frequency)

In [ ]:
dataset.shape

In [ ]:
plt.imshow(np.min(dataset, axis=0)/255.)

In [ ]:
df = pd.DataFrame(dataset.reshape(dataset.shape[0], -1))
df.shape

In [ ]:
window_size = 14

In [ ]:
%time new = df.rolling(window_size).quantile(0.05)

In [ ]:
%time ok = new.to_numpy().reshape(dataset.shape).astype('uint8')

In [ ]:
plt.imshow(ok[18])

In [ ]:
%time mimwrite('gif_orig.gif', dataset, format='gif', fps=14)

In [ ]:
%time mimwrite('gif_ok.gif', ok, format='gif', fps=14)

In [ ]:
X = np.transpose(dataset.reshape((dataset.shape[0], -1)))

In [ ]:
from sklearn.decomposition import FastICA
transformer = FastICA(n_components=14, random_state=0)
X_transformed = transformer.fit_transform(X)
X_transformed.shape

In [ ]:
components = X_transformed.reshape((*dataset.shape[1:], -1))
components.shape

In [ ]:
components.min(axis=(-2, -1)).shape

In [ ]:
rgb = components.copy()
rgb -= rgb.min()
rgb /= rgb.max()

In [ ]:
plt.imshow(rgb[..., 0])

In [ ]:
plt.imshow(rgb[..., 1])

In [ ]:
plt.imshow(rgb[..., 2])

In [ ]:
plt.imshow(rgb[..., 3])

In [ ]:
plt.imshow(rgb[..., 4])